# Realistic Profiling Example

In [ ]:
import math

import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
import numpy as np

plt.style.use(["nature", "notebook", {"image.cmap": "inferno"}])

## Goal: Write function to smooth an image by a Gaussian kernel

This is just implementing a _Gaussian blur_, where $\hat I$ is the smoothed image with pixel values $\hat I_{i,j}$, and the source image is $I$, and the kernel is $K$, a smaller 2D (s by s) array representing the kernel with pixel values $K_{m,n}$.  The $\circledast$ symbol represent the _convolution_ operator. 

$$
\begin{align*}
\hat I_{i,j}
=& (I \circledast K)_{i,j}\\
=& \sum_{i=0}^{h} \sum_{j=0}^{w} \sum_{m=0}^{s} \sum_{n=0}^{s} I_{i-m,\,j-n}\, K_{m,n}
\end{align*}
$$


In the actual implementation, we have to take into account some edge effects, so we add some padding 

## Setup: the image

Let's load up a greyscale image.  In this example, we are using an image from the Chandra space telescope of the supernova remnant _Cassiopia A_.  However, you can try others!

In [ ]:
image_full = np.load("cas_a.npy").astype(np.float64)
image = image_full[300:500, 300:500]
print("image_full: ", image_full.shape)
print("     image: ", image.shape)

Some fun facts:
* You are seeing the explosion shell of a supernova (and exploding star) in our galaxy, that exploded in the mid 17th Century CE.
* This image is in X-ray wavelengths, but it is visible in radio, optical, X-ray, and gamma-ray wavelengths.

In [ ]:
plt.imshow(image_full, origin="lower", norm=PowerNorm(0.3))
plt.vlines([300,500],0,1000)
plt.hlines([300,500],0,1000)

In [ ]:
plt.imshow(image, origin="lower", norm=PowerNorm(0.3))
plt.colorbar()

## Setup: The Kernel

In [ ]:
def gaussian_kernel(sigma, size):
    """Create a 2D Gaussian kernel"""
    kernel = np.zeros((size, size))
    center = size // 2

    for i in range(size):
        for j in range(size):
            x = i - center
            y = j - center
            kernel[i, j] = math.exp(-(x**2 + y**2) / (2 * sigma**2))

    kernel = kernel / kernel.sum()
    return kernel

In [ ]:
kernel = gaussian_kernel(2.0, 11)

plt.figure(figsize=(1,1))
plt.imshow(kernel)

First we'll set up some useful functions, one to plot the result and one to compare performances.  

In [ ]:
def plot(before, after):
    """Plot the original and smoothed image"""
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].imshow(before, origin="lower", norm=PowerNorm(0.3))
    ax[0].set_title("Original")

    ax[1].imshow(after, origin="lower", norm=PowerNorm(0.3))
    ax[1].set_title("Smoothed")

## First implementation: python for-loops 

In [ ]:
np.zeros_like(image, dtype=float)

In [ ]:
result = [[0]*image.shape[0]]*image.shape[1]

In [ ]:
def smooth_loops(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """Smooth image with kernel, using simple python math."""
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    result = [0.0] * (h * w)

    for i in range(h):
        for j in range(w):
            value = 0.0
            for m in range(kernel_size):
                for n in range(kernel_size):
                    ii = i + m - pad
                    jj = j + n - pad
                    ii = max(0, min(h - 1, ii))
                    jj = max(0, min(w - 1, jj))
                    value += image[ii, jj] * kernel[m, n]

            result[i + w*j] = value

    return np.asarray(result).reshape(image.shape)  # the only bit of numpy used to turn back into 2D

In [ ]:
image_smooth = smooth_loops(image, kernel)
plot(image, image_smooth)

##  Function profiling:

There is a built-in "magic" function of Jupyter for running a cProfile profile on a statement.  See the lecture notes on how to run this outside of a notebook.

In [ ]:
%prun smooth_loops(image, kernel)

## Line Profiling

Looking at function calls doesn't give the full picture here. We just see that "min" and "max" are called many times . Let's see where the slowest part of `smooth_loops` is **by line**.

First we need to load the extension for Jupyter:

In [ ]:
%load_ext line_profiler

Now, we can use the magic %lprun Jupyter function to run the line profiler on the function we want.  Note that you can also do this outside of a notebook, see the lecture notes.

In [ ]:
%lprun -f smooth_loops smooth_loops(image, kernel)


<div style="color:red; font-size:large">
STOP HERE, Back to slides
</div>


# Optimization

Now, let's try different implementations of the Gaussian Blur code, and **compare them**

To store my performance results, I create a dictionary called `PERF` that will hold the results by name. That's what the `plot_performance()` function expects as input

In [ ]:
PERF = dict()

And now we do the first measurement (note we use `%timeit -o` to get "output", which is an object containing some stats

In [ ]:
PERF["loops"] = %timeit -o smooth_loops(image, kernel)

In [ ]:
PERF

Let's also make a nice function to plot the results

In [ ]:
def plot_speed(perf: dict[str], absolute=True, ax=None):
    """
    Compare Performance between several timeit measurements.

    Parameters
    ----------
    perf: dict
        dictionary of timeit results by name
    absolute: bool
        if True, show speeds, if False show relative speedup compared
        to the first trial
    ax: plt.Axis | None
        if set, plot on this axis. Otherwise use current.
    """
    ax = ax or plt.gca()
    names = [n.replace(" ", "\n") for n in perf.keys()]
    x = list(range(len(names)))
    ax.set_xticks(x, names)

    averages = [m.average for m in perf.values()]
    std = [m.stdev for m in perf.values()]

    if absolute:
        y = averages
        ax.errorbar(x=x, y=y, yerr=std, marker="o", linestyle="none", markersize=10)
        ax.set_ylabel("Average Execution Speed [s]")
        ax.set_yscale("log")
        ax.set_title("Lower is better")
    else: 
        y = averages[0]/np.array(averages)
        ax.bar(x=x, height=y)  
        ax.set_ylabel(f"Speedup relative to {names[0]}")
        ax.set_title("Higher is better")

   
    
    ax.grid(True)

def plot_performance(perf: dict[str]):
    """
    Plot performance and relative performance 
    Parameters
    ----------
    perf: dict
        dictionary of timeit results by name
    """
    fig, ax = plt.subplots(1,2, figsize=(20,5))
    plot_speed(perf=perf, absolute=True, ax=ax[0])
    plot_speed(perf=perf, absolute=False, ax=ax[1])


In [ ]:
plot_performance(PERF)

### Second implementation: using NumPy to avoid some loops

In [ ]:
def smooth_numpy_outerloop(image, kernel):
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    
    # Pad the image
    padded = np.pad(image, pad, mode='edge')
    result = np.zeros_like(image, dtype=float)
    
    for m in range(kernel_size):
        for n in range(kernel_size):
            result += padded[m:m+h, n:n+w] * kernel[m, n]
    
    return result

In [ ]:
image_smooth = smooth_numpy_outerloop(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF['numpy outer-loop'] = %timeit -o smooth_numpy_outerloop(image, kernel)

In [ ]:
plot_performance(PERF)

### Third implementation: Avoid all loops, advanced Numpy

This is one way to use no explicit loops, using some really esoteric tricks of numpy indexing.  Note that is no longer very readable!

In [ ]:
from numpy.lib.stride_tricks import as_strided

def smooth_numpy_noloops(image, kernel):
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    
    # Pad the image
    padded = np.pad(image, pad, mode='edge')

    # Use as_strided to create sliding windows
    shape = (h, w, kernel_size, kernel_size)
    strides = padded.strides + padded.strides
    windows = as_strided(padded, shape=shape, strides=strides)
    
    # Multiply windows by kernel and sum
    result = np.sum(windows * kernel, axis=(2, 3))
    
    return result

In [ ]:
image_smooth = smooth_numpy_noloops(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF["numpy no-loops"] = %timeit -o smooth_numpy_noloops(image, kernel)

In [ ]:
plot_performance(PERF)

This actually is slower! And in fact uses way more memory!  (nice test for memory_profiler)

### Fourth Implementation: Using Numba to compile!

Numba compiles a python function into machine code.

This is nice when you don't want to deal with numpy vectors and you prefer to keep it as loops:

In [ ]:
from numba import jit

In [ ]:
@jit  # <--- the magic happens here
def smooth_loops_jit(image, kernel):
    """Smooth image with kernel, using simple python math."""
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    result = [0.0] * (h * w)

    for i in range(h):
        for j in range(w):
            value = 0.0
            for m in range(kernel_size):
                for n in range(kernel_size):
                    ii = i + m - pad
                    jj = j + n - pad
                    ii = max(0, min(h - 1, ii))
                    jj = max(0, min(w - 1, jj))
                    value += image[ii, jj] * kernel[m, n]

            result[i + w*j] = value

    return np.asarray(result).reshape(image.shape)  # the only bit of numpy used to turn back into 2D

In [ ]:
image_smooth = smooth_loops_jit(image, kernel)
plot(image, image_smooth)

In [ ]:
smooth_loops_jit

In [ ]:
PERF["loops jit"] = %timeit -o smooth_loops_jit(image, kernel)

In [ ]:
plot_performance(PERF)

### Fifth Implementation: use an existing library function

This is the simplest - we realize that scipy already implements this algorithm, and we can be pretty sure that they have done it in the most performant way, and this results in the simplest code, though it's also not easy to see what actually happens inside.

In [ ]:
from scipy.ndimage import convolve

def smooth_scipy(image, kernel):
    """Smooth image by kernel using a built-in scipy function."""
    return convolve(image, kernel, mode="nearest")

In [ ]:
image_smooth = smooth_scipy(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF["scipy"] = %timeit -o smooth_scipy(image, kernel)

In [ ]:
plot_performance(PERF)

Also note, from `help(convolve)`:

> `convolve` has experimental support for **Python Array API Standard** compatible
backends in addition to NumPy. Please consider testing these features
by setting an environment variable ``SCIPY_ARRAY_API=1`` and providing
CuPy, PyTorch, JAX, or Dask arrays as array arguments. The following
combinations of backend and device (or other capability) are supported.

```
====================  ====================  ====================
Library               CPU                   GPU
====================  ====================  ====================
NumPy                 ✅                     n/a                 
CuPy                  n/a                   ✅                   
PyTorch               ✅                     ⛔                   
JAX                   ⚠️ no JIT             ⛔                   
Dask                  ⚠️ computes graph     n/a                 
====================  ====================  ====================
```

### Sixth implementation: Writing it in C and interfacing?

What about skipping python, writing it in C and calling the C implementation from python?
There are a few ways to do this, but here is one pretty simple one.

**NOTE** this one won't work if you don't have `gcc` installed on your computer.  You can try other compilers though!

In [ ]:
C_CODE="""
void convolve2d(double *image, int h, int w, 
                double *kernel, int kernel_size, 
                double *result) {
    int pad = kernel_size / 2;
    
    for (int i = 0; i < h; i++) {
        for (int j = 0; j < w; j++) {
            double value = 0.0;
            
            for (int m = 0; m < kernel_size; m++) {
                for (int n = 0; n < kernel_size; n++) {
                    int ii = i + m - pad;
                    int jj = j + n - pad;
                    
                    // Clamp to image boundaries
                    if (ii < 0) ii = 0;
                    if (ii >= h) ii = h - 1;
                    if (jj < 0) jj = 0;
                    if (jj >= w) jj = w - 1;
                    
                    value += image[ii * w + jj] * kernel[m * kernel_size + n];
                }
            }
            
            result[i * w + j] = value;
        }
    }
}
"""
with open("convolve.c", "w") as output:
    output.write(C_CODE)

Here we have to run a bash shell command to compile the code:

* you can substitute `clang` for `gcc` if you have that compiler instead (e.g. on a mac)
* we set `-O2` to tell the compiler we want level-2 optimization. Try without it, or with `-O0`


In [ ]:
%%bash
gcc -shared -fPIC -O2 -o libconvolve.so convolve.c
ls -l ./*.so

In [ ]:
import ctypes
lib = ctypes.CDLL('./libconvolve.so') 

# have to set the argument types to interface Numpy and C:
lib.convolve2d.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]
lib.convolve2d.restype = None

In [ ]:
# since the C function cannot return an array, we allocate one here and pass it:
# it has to be a 1D array (C doesn't have 2D arrays), so we make it the same as the flattened image)
result = np.zeros_like(image.flatten(), dtype=np.float64)
lib.convolve2d(image.ravel(), image.shape[0], image.shape[1],kernel.ravel(), kernel.shape[0],result)
plot(image, result.reshape(image.shape))

In [ ]:
PERF["C"] = %timeit -o lib.convolve2d(image.ravel(), image.shape[0], image.shape[1],kernel.ravel(), kernel.shape[0],result)

In [ ]:
plot_performance(PERF)

In [ ]:
PERF_SORTED = {k: v for k, v in reversed(sorted(PERF.items(), key=lambda item: item[1].average))}
plot_performance(PERF_SORTED)


<div style="color:red; font-size:large">
STOP HERE, Back to slides
</div>

# Memory Profiling

## Memory Profiling with memory_profiler

* Now let's see which uses more memory, to get better stats let's run on the full image (*much* larger)
* we won't run `smooth_loops` as it is too slow for this demo... (but try it yourself, it should allocate very little memory)

In [ ]:
%load_ext memory_profiler

In [ ]:
MEM = dict()
MEM["loops"] = %memit -o smooth_loops(image_full, kernel)
MEM["numpy outer-loop"] = %memit -o smooth_numpy_outerloop(image_full, kernel)
MEM["numpy no-loops"] = %memit -o smooth_numpy_noloops(image_full, kernel)
MEM["scipy"] = %memit -o smooth_scipy(image_full, kernel)

In [ ]:
def plot_memory(mem: dict[str]):
    names = [n.replace(" ","\n") for n in mem.keys()]
    peak = [max(m.mem_usage) for m in mem.values()]
    x = list(range(len(names)))

    plt.scatter(x=x, y=peak, marker="o", s=100)
    plt.xticks(x, names)
    plt.ylabel("Memory Usage")
    plt.grid()

In [ ]:
plot_memory(MEM)

**Conclusion**: It's not always best to eliminate all loops! not always faster, and can use much more memory!


## Let's also try another package: **memray** 

**note will not work on Windows!**


In [ ]:
%load_ext memray

In [ ]:
%%memray_flamegraph --temporary-allocations
smooth_numpy_basic(image_full, kernel)

In [ ]:
%%memray_flamegraph --temporary-allocations
smooth_numpy_noloops(image_full, kernel)

In [ ]:
%memit smooth_numpy_basic(image, kernel)

### A memory leak

In [ ]:
%%memray_flamegraph
import numpy as np

SOME_GLOBAL_STATE = []

def my_leaky_function():
    SOME_GLOBAL_STATE.append(np.ones((1000,1000)))

for ii in range(1000):
    my_leaky_function()